In [1]:
import add_packages
import config
from pprint import pprint

from my_langchain import (
    document_loaders, text_splitters, text_embedding_models, vector_stores,
    chat_models, prompts, utils, output_parsers, agents, documents, runnables,
    llms, histories
)

# Get started

## prompt + model + output parser

In [ ]:
# Create a chain that takes a topic and generates a joke.

# Takes in a dictionary of template variables and produces a PromptValue. 
# PromptValue is a wrapper around a completed prompt that can be passed to 
# either an LLM or ChatModel. 
# It defines logic for producing BaseMessages and a string.
prompt = prompts.ChatPromptTemplate.from_template(
  "Tell me a joke about {topic}"
)

# The PromptValue is passed to model. 
# ChatModel outputs a BaseMessage. LLM outputs string.
model = chat_models.chat_openai

# Pass model output to output_parser, a BaseOutputParser inputs string or 
# BaseMessage and converts to string.
output_parser = output_parsers.str_output_parser()

# chains together different components feeds the output from one component as 
# input into the next component.

# user input passed to prompt template, prompt template output passed to model,
# model output passed to output parser.
chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

In [ ]:
prompt_value = prompt.invoke({"topic": "ice cream"})
pprint(prompt_value)
pprint(prompt_value.to_messages())
pprint(prompt_value.to_string())

In [ ]:
model.invoke(prompt_value)

## RAG Search Example

In [ ]:
vectorstore = vector_stores.chroma.Chroma.from_texts(
  ["harrison worked at kensho", "bears like to eat honey"],
  embedding=text_embedding_models.openai_embeddings()
)

# In memory store to retrieve documents based on a query. 
# Runnable component can be chained with other components or run separately.
retriever = vectorstore.as_retriever() # .invoke("")

# The prompt template takes in context and question as values. 
# Before building the prompt template, relevant documents should be retrieved 
# for inclusion in the context.
template = """\
Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = prompts.ChatPromptTemplate.from_template(template)

model = chat_models.chat_openai
output_parser = output_parsers.str_output_parser()

# Prepare inputs for the prompt with retrieved documents and user question, 
# retriever for document search, RunnablePassthrough for user's question.
setup_and_retrieval = runnables.RunnableParallel(
  {"context": retriever, "question": runnables.RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

In [ ]:
chain.invoke("where did harrison work?")

# Interface

In [ ]:
# PromptTemplate + ChatModel chain.

model = chat_models.chat_openai
prompt = prompts.ChatPromptTemplate.from_template(
  "Tell me a joke about {topic}"
)
chain = prompt | model

## Input Schema

A description of the inputs accepted by a Runnable. 

Pydantic model generated from the structure of any Runnable. 

Call .schema() to obtain a JSONSchema representation.


In [ ]:

chain.input_schema.schema()

## Output Schema

## Stream

In [ ]:
for s in chain.stream({"topic": "bears"}):
  print(s.content, end="", flush=True)

## Invoke

In [ ]:
chain.invoke({"topic": "bears"})

## Batch

In [ ]:
# Set the number of concurrent requests using the max_concurrency parameter.
chain.batch([
  {"topic": "bears"},
  {"topic": "cats"},
], config={"max_concurrency": 5})

## Parallelism

LangChain Expression Language supports parallel requests by executing each 
element in parallel when using a RunnableParallel.

In [ ]:

template1 = "tell me a joke about {topic}"
template2 = "write a short (2 line) poem about {topic}"
chain1 = prompts.ChatPromptTemplate.from_template(template1) | model
chain2 = prompts.ChatPromptTemplate.from_template(template2) | model
combined = runnables.RunnableParallel(joke=chain1, poem=chain2)
combined.invoke({"topic": "bears"})

# How to

## Manipulating inputs & output - RunnableParallel

RunnableParallel manipulating output of one Runnable match input format of next 
Runnable in sequence.

In [ ]:


vectorstore = vector_stores.chroma.Chroma.from_texts(
  ["harrison worked at kensho"], embedding=text_embedding_models.openai_embeddings()
)
retriever = vectorstore.as_retriever()

# The input to prompt is a map with keys “context” and “question”. 
# Retrieve the context using the retriever and pass through the user input 
# under the “question” key.
template = """\
Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = prompts.ChatPromptTemplate.from_template(template)

model = chat_models.chat_openai

retrieval_chain = (
  {
    "context": retriever,
    "question": runnables.RunnablePassthrough()
  }
  | prompt
  | model
  | output_parsers.str_output_parser()
)

retrieval_chain.invoke("where did harrison work?")

### itemgetter

Extract data from the map using Python's itemgetter with RunnableParallel.

itemgetter is used to extract specific keys from the map.

In [ ]:

from operator import itemgetter

template = """\
Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = prompts.ChatPromptTemplate.from_template(template)

chain = (
  {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question"),
    "language": itemgetter("language"),
  }
  | prompt 
  | model
  | output_parsers.StrOutputParser()
)

chain.invoke({
  "question": "where did harrison work",
  "language": "italian"
})

### Parallelize Runnables
RunnableParallel (aka. RunnableMap) executes multiple Runnables in parallel and returns their output as a map.

In [ ]:

model = llms.llm_openai

template1 = "tell me a joke about {topic}"
template2 = "write a 2-line poem about {topic}"
chain1 = prompts.ChatPromptTemplate.from_template(template1) | model
chain2 = prompts.ChatPromptTemplate.from_template(template2) | model

map_chain = runnables.RunnableParallel(Chain1=chain1, Chain2=chain2)
map_chain.invoke({"topic": "bear"})

## Passing data through - RunnablePassthrough

RunnablePassthrough allows passing inputs unchanged or with extra keys, used with RunnableParallel to assign data to a new key in the map. 

RunnablePassthrough() pass input through. 

RunnablePassthrough called with assign will take the input and add the extra arguments passed to the assign function.


In [ ]:

runnable = runnables.RunnableParallel(
  # passed key was called with RunnablePassthrough() and it passed on {'num': 1}.
  passed=runnables.RunnablePassthrough(),
  # multiplies the numerical value by 3
  extra=runnables.RunnablePassthrough.assign(mult=lambda x: x["num"]*3),
  # set single value adding 1 to num
  modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

### Retrieval

The input is a map with keys “context” and “question”. The user input is the question. Retrieve the context and pass the user input as the question key. The RunnablePassthrough allows passing the user’s question to the prompt and model.


In [ ]:

vectorstore = vector_stores.chroma.Chroma.from_texts(
  ["harrison worked at kensho"],
  embedding=text_embedding_models.openai_embeddings()
)
retriever = vectorstore.as_retriever()

template = """\
Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = prompts.ChatPromptTemplate.from_template(template)

model = chat_models.chat_openai

chain = (
  {
    "context": retriever, "question": runnables.RunnablePassthrough()
  }
  | prompt
  | model
  | output_parsers.str_output_parser()
)

chain.invoke("where did harrison work?")

## Run Custom Functions - RunnableLambda

Use arbitrary functions in the pipeline.

All inputs to functions must be a single argument. If a function accepts multiple arguments, create a wrapper that accepts a single input and unpacks it.

Optional RunnableConfig can be used by Runnable lambdas to pass callbacks, tags, and configuration information to nested runs.


In [ ]:
from operator import itemgetter

def length_function(text):
  return len(text)

def _multiple_length_function(text1, text2):
  return len(text1)*len(text2)

def multiple_length_function(_dict):
  return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = prompts.ChatPromptTemplate.from_template("what is {a} + {b}")
model = chat_models.chat_openai

chain = (
  {
    "a": itemgetter("foo") | runnables.RunnableLambda(length_function),
    "b": {
      "text1": itemgetter("foo"),
      "text2": itemgetter("bar"),
    } | runnables.RunnableLambda(multiple_length_function)
  }
  | prompt
  | model
)

chain.invoke({"foo": "bar", "bar": "gah"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig
import json
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\n```text\n{input}\n```\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

## Dynamically route logic based on input - RunnableBranch

Routing creates non-deterministic chains based on previous step output, providing structure and consistency for LLM interactions.

Two ways to perform routing: Conditionally return runnables from a RunnableLambda or using a RunnableBranch.



### Setup

Illustrate both methods using a two step sequence. The first step classifies an input question as LangChain, Anthropic, or Other, then routes to a corresponding prompt chain.


In [ ]:
# Create a chain to identify incoming questions as LangChain, Anthropic, or Other.
template = """\
Given the user question below, classify it as either being about `LangChain`, \
`Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:\
"""
prompt = prompts.PromptTemplate.from_template(template)

chain = (
  prompt
  | chat_models.chat_openai
  | output_parsers.StrOutputParser()
)

chain.invoke({"question": "how do I call Anthropic?"})

### Custom Function

In [ ]:
import typing


# create three sub chains

template_langchain = """\
You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:\
"""
prompt_langchain = prompts.PromptTemplate.from_template(template_langchain)
chain_langchain = prompt_langchain | chat_models.chat_openai

template_anthropic = """\
You are an expert in anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:\
"""
prompt_anthropic = prompts.PromptTemplate.from_template(template_anthropic)
chain_anthropic = prompt_anthropic | chat_models.chat_openai

template_general = """\
Respond to the following question:

Question: {question}
Answer:\
"""
prompt_general = prompts.PromptTemplate.from_template(template_general)
chain_general = prompt_general | chat_models.chat_openai

# Use a custom function to route between different outputs
def route(info: typing.Dict[str, str]):
  if "anthropic" in info["topic"].lower():
    return chain_anthropic
  elif "langchain" in info["topic"].lower():
    return chain_langchain
  else:
    return chain_general

chain_full = (
  {
    "topic": chain,
    "question": lambda x: x["question"]
  }
  | runnables.RunnableLambda(route)
)

In [ ]:
print(chain_full.invoke({"question": "how do I use Anthropic?"}))
print(chain_full.invoke({"question": "how do I use LangChain?"}))
print(chain_full.invoke({"question": "whats 2 + 2?"}))


## Bind runtime args

Invoke a Runnable within a Runnable sequence with constant arguments that are not part of the output of the preceding Runnable in the sequence, and not part of the user input. Use Runnable.bind() to pass these arguments.


In [ ]:
# Prompt + model sequence
prompt = prompts.ChatPromptTemplate.from_messages([
  (
    "system",
    "Write out the following equation using algebraic symbols then solve it. \
     Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
  ),
  (
    "human",
    "{equation_statement}"
  )
])

model = chat_models.chat_openai
runnable = (
  {
    "equation_statement": runnables.RunnablePassthrough()
  }
  | prompt
  # call the model with certain stop words
  | model.bind(stop="SOLUTION")
  # | model
  | output_parsers.StrOutputParser()
)

In [ ]:
pprint(runnable.invoke("x raised to the third plus seven equals 12"))

### Attaching OpenAI functions

Attach OpenAI functions to a compatible OpenAI model.

In [ ]:
function = {
    "name": "solver",
    "description": "Formulates and solves an equation",
    "parameters": {
        "type": "object",
        "properties": {
            "equation": {
                "type": "string",
                "description": "The algebraic expression of the equation",
            },
            "solution": {
                "type": "string",
                "description": "The solution to the equation",
            },
        },
        "required": ["equation", "solution"],
    },
}

prompt = prompts.ChatPromptTemplate.from_messages([
    (
        "system",
        "Write out the following equation using algebraic symbols then solve it. \
     Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
    ),
    (
        "human",
        "{equation_statement}"
    )
])

model = chat_models.chat_openai.bind(
    function_call={"name": "solver"}, functions=[function]
)

runnable = (
    {
        "equation_statement": runnables.RunnablePassthrough()
    }
    | prompt
    | model
)

pprint(runnable.invoke("x raised to the third plus seven equals 12"))

### Attaching OpenAI tools

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

model = chat_models.chat_openai.bind(tools=tools)
model.invoke("What's the weather in SF, NYC and LA?")

## Configure chain internals at runtime

Two methods for experimenting with and exposing multiple ways of doing things to the end user.

- configurable_fields method. Configure particular fields of a runnable.
- configurable_alternatives method to list alternatives for any runnable set during runtime.

### Configuration Fields


#### With LLMs

LLMs configure temperature.

In [ ]:
model = chat_models.chat_openai.configurable_fields(
  temperature=runnables.ConfigurableField(
    id="llm_temperature",
    name="LLM Temperature",
    description="The temperature of the LLM",
  )
)

In [ ]:
model.invoke("pick a random number")

In [ ]:
model.with_config(configurable={"llm_temperature": 0.9}).invoke(
  "pick a random number"
)

In [ ]:
# used as part of a chain
template = "Pick a random number above {x}"
prompt = prompts.PromptTemplate.from_template(template)
chain = prompt | model

print(chain.invoke({"x": 0}))
print(chain.with_config(
  configurable={"llm_temperature": 0.9}
).invoke({"x": 0}))

#### With HubRunnables

Useful for switching prompts.

In [ ]:
prompt = runnables.HubRunnable("rlm/rag-prompt").configurable_fields(
  owner_repo_commit=runnables.ConfigurableField(
    id="hub_commit",
    name="Hub Commit",
    description="The Hub commit to pull from"
  )
)

print(prompt.invoke({"question": "foo", "context": "bar"}))
print(prompt.with_config(
  configurable={"hub_commit": "rlm/rag-prompt-llama"}
).invoke({"question": "foo", "context": "bar"}))

### Configurable Alternatives


#### With LLMs


In [ ]:
llm = chat_models.chat_openai.configurable_alternatives(
  runnables.ConfigurableField(id="llm"), # Field with an ID for configuration
  default_key="openai",  # Default LLM key
  anthropic=chat_models.chat_anthropic, # option
  gpt4=chat_models.ChatOpenAI(model="gpt-4") # option
  # Add more configuration options here
)

template = "Tell me a joke about {topic}"
prompt = prompts.PromptTemplate.from_template(template)

chain = prompt | llm

In [ ]:
print(chain.invoke({"topic": "bears"}))

In [ ]:
print(chain.with_config(
  configurable={"llm": "gpt4"}
).invoke({"topic": "bears"}))

#### With Prompts

Alternate between prompts


In [ ]:
llm = chat_models.chat_openai

template_joke = "Tell me a joke about {topic}"
template_poem = "Write a short poem about {topic}"
prompt = prompts.PromptTemplate.from_template(template_joke)\
  .configurable_alternatives(
    # Field with an ID for configuration
    runnables.ConfigurableField(id="prompt"),
    default_key="joke",
    poem=prompts.PromptTemplate.from_template(template_poem),  # Option: poem
    # Add more configuration options here
  )
  
chain = prompt | llm

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
chain.with_config(
  configurable={"prompt": "poem"}
).invoke({"topic": "bears"})

#### With Prompts and LLMs

Multiple things configurable with prompts and LLMs.

In [ ]:
llm = chat_models.chat_openai.configurable_alternatives(
  runnables.ConfigurableField(id="llm"), # Field with an ID for configuration
  default_key="openai",  # Default LLM key
  anthropic=chat_models.chat_anthropic, # option
  gpt4=chat_models.ChatOpenAI(model="gpt-4") # option
  # Add more configuration options here
)

template_joke = "Tell me a joke about {topic}"
template_poem = "Write a short poem about {topic}"
prompt = prompts.PromptTemplate.from_template(template_joke)\
  .configurable_alternatives(
    # Field with an ID for configuration
    runnables.ConfigurableField(id="prompt"),
    default_key="joke",
    poem=prompts.PromptTemplate.from_template(template_poem),  # Option: poem
    # Add more configuration options here
  )

chain = prompt | llm

In [ ]:
chain.with_config(
  configurable={
    "prompt": "poem",
    "llm": "gpt4"
  }
).invoke(
  {
    "topic": "bears",
  }
)

#### Saving configurations

Save configured chains as objects.

In [ ]:
chain_gpt4_joke = chain.with_config(
  configurable={
    "prompt": "poem",
    "llm": "gpt4"
  }
)

## Create a runnable with the `@chain` decorator

Turn an arbitrary function into a chain by adding a @chain decorator. Equivalent to wrapping in a RunnableLambda.

Improved observability by tracing chain correctly. Calls to runnables inside function traced as nested children.

Allow use as runnable, compose in chain.

In [ ]:
template1 = "Tell me a joke about {topic}"
prompt1 = prompts.ChatPromptTemplate.from_template(template1)
template2 = "What is the subject of this joke: {joke}"
prompt2 = prompts.ChatPromptTemplate.from_template(template2)

@runnables.chain
def custom_chain(text):
  prompt_val1 = prompt1.invoke({"topic": text})
  output1 = chat_models.chat_openai.invoke(prompt_val1)
  parsed_output1 = output_parsers.StrOutputParser().invoke(output1)
  
  chain2 = prompt2 | chat_models.chat_openai | output_parsers.StrOutputParser()
  return chain2.invoke({"joke": parsed_output1})

# Check LangSmith traces for custom_chain trace with calls to OpenAI nested underneath.
custom_chain.invoke("bears")

## Add fallbacks


### Handling LLM API Errors


#### Specifying errors to handle


### Fallbacks for Sequences

## Stream custom generator functions

Use generator functions (yield) in a LCEL pipeline.

The signature of generators: Iterator[Input] -> Iterator[Output]. 

For async generators: AsyncIterator[Input] -> AsyncIterator[Output].

useful for: 

- implementing custom output parser
- modifying output of previous step, preserving streaming capabilities

Implement custom output parser for comma-separated lists.

### Sync version


In [ ]:
from typing import Iterator, List

template = "Write a comma-separated list of 5 animals simiar to: {animal}" 
prompt = prompts.ChatPromptTemplate.from_template(template)
model = chat_models.chat_openai
chain = prompt | model | output_parsers.StrOutputParser()


In [ ]:
for chunk in chain.stream({"animal": "bear"}):
  print(chunk, end="", flush=True)

In [ ]:
# Custom parser splits an iterator of llm tokens into a list of strings 
# separated by commas
def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
  # Hold partial input until we get a comma
  buffer = ""
  for chunk in input:
    # Add the current chunk to the buffer
    buffer += chunk
    # While there are commas in the buffer
    while "," in buffer:
      # Split the buffer on the comma
      comma_index = buffer.index(",")
      # Yield everything before the comma
      yield [buffer[:comma_index].strip()]
      # Save the rest for the next iteration
      buffer = buffer[comma_index + 1:]
  # Yield the last chunk
  yield [buffer.strip()]

list_chain = chain | split_into_list

for chunk in list_chain.stream({"animal": "bear"}):
  print(chunk, end="", flush=True)

### Async version

## Inspect runnables

Inspecting a runnable with LCEL can help understand what is happening. 


In [ ]:
# LCEL does retrieval.
vectorstore = vector_stores.chroma.Chroma.from_texts(
  ["harrison worked at kensho"], embedding=text_embedding_models.OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """\
Answer the question based only on the following context:
{context}

Question: {question}\
"""
prompt = prompts.ChatPromptTemplate.from_template(template)

model = chat_models.chat_openai

chain = (
  {
    "context": retriever, "question": runnables.RunnablePassthrough()
  }
  | prompt
  | model
  | output_parsers.StrOutputParser()
)


In [ ]:
# Get a graph of the runnable
chain.get_graph().print_ascii()

In [5]:
# Get the prompts present in the chain.
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}'))])]

## Add message history (memory)

The RunnableWithMessageHistory adds message history to chain by wrapping another Runnable and managing chat message history.

Used for any Runnable that takes as input 

- a sequence of BaseMessage
- a dict with a key that takes a sequence of BaseMessage
- a dict with a key that takes the latest message(s) as a string or sequence of BaseMessage, and a separate key that takes historical messages.

Output: Return a string content of AIMessage, sequence of BaseMessage, or dict with a key containing a sequence of BaseMessage.

### Setup

Construct a runnable (accepts a dict as input and returns a message as output).

To manage message history, need: 
- Runnable; 
- Callable returning instance of BaseChatMessageHistory.

Demonstrate using in-memory ChatMessageHistory and persistent storage using RedisChatMessageHistory.

In [2]:
model = chat_models.chat_openai
template_msg = [
  (
    "system",
     "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
  ),
  prompts.MessagesPlaceholder(variable_name="history"),
  (
    "human",
    "{input}",
  )
]
prompt = prompts.ChatPromptTemplate.from_messages(template_msg)
runnable = prompt | model

### In-memory



In [3]:

# Chat history is stored in memory using a global Python dictionary.
store = {}

def get_session_history(
  user_id: str, conversation_id: str
) -> histories.BaseChatMessageHistory:
  """
  Callable references a dict to return an instance of ChatMessageHistory. 
  
  The arguments can be specified by passing a configuration to the 
  RunnableWithMessageHistory at runtime. 
  
  The configuration parameters for tracking message histories can be customized 
  by passing a list of ConfigurableFieldSpec objects to the 
  history_factory_config parameter. 
  
  Two parameters used are user_id and conversation_id.
  """
  if (user_id, conversation_id) not in store:
    store[(user_id, conversation_id)] = histories.ChatMessageHistory()
  return store[(user_id, conversation_id)]

with_message_history = runnables.RunnableWithMessageHistory(
  runnable,
  get_session_history,
  input_messages_key="input",  # latest input message
  history_messages_key="history",  # key to add historical messages to
  history_factory_config=[
    runnables.ConfigurableFieldSpec(
      id="user_id", annotation=str, name="User ID", default="",
      description="Unique identifier for the user.", is_shared=True,
    ),
    runnables.ConfigurableFieldSpec(
      id="conversation_id", annotation=str, name="Conversation ID", default="", 
      description="Unique identifier for the conversation.", is_shared=True,
    ),
  ]
)

# When invoking new runnable, specify corresponding chat history via 
# configuration parameter.
result1 = with_message_history.invoke(
  { "ability": "math", "input": "What does cosine mean?" },
  config={ "configurable": { "user_id": "123", "conversation_id": "1" } }
)
result2 = with_message_history.invoke(
  { "ability": "math", "input": "What?" },
  config={ "configurable": { "user_id": "123", "conversation_id": "1" } }
)

print(result1.content)
print(result2.content)

Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.
Cosine is a math function that helps find the ratio of the adjacent side to the hypotenuse in a triangle.


In [9]:
# continue

Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.


#### Examples with runnables of different signatures


#### Persistent storage


### Setup


#### LangSmith

# Cookbook

## Prompt + LLM


## RAG


## Multiple chains


## Querying a SQL DB


## Agents


## Code writing


## Routing by semantic similarity


## Adding memory


## Adding moderation


## Managing prompt size


## Using tools